## Dataset de revisiones físicas

### Limpieza inicial

In [1]:
import pandas as pd

df_check = pd.read_csv('../Dataset/datos_check.csv')

In [2]:
df_check

,negocio_id,fecha
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."
...,...,...
161945,zzvlwkcNR1CCqOPXwuvz2A,"2017-05-06 20:05:15, 2017-05-12 22:37:03, 2017..."
161946,zzwaS0xn1MVEPEf0hNLjew,"2010-02-16 02:09:56, 2010-07-05 05:40:48, 2010..."
161947,zzwhN7x37nyjP0ZM8oiHmw,"2016-03-06 13:27:02, 2016-03-09 00:41:53, 2016..."
161948,zzwicjPC9g246MK2M1ZFBA,"2012-09-22 00:26:15, 2012-09-23 20:12:00, 2012..."


### Búsqueda de valores nulos

In [3]:
print("¿Hay valores nulos?\n", pd.isnull(df_check).any(), "\n")
print("¿Cuantos?\n", pd.isnull(df_check).sum(), "\n")

¿Hay valores nulos?
 negocio_id    False
fecha         False
dtype: bool 

¿Cuantos?
 negocio_id    0
fecha         0
dtype: int64 



### Comprobación sintáctica de la fecha

No se imprime ningún error por lo que, aparentemente, todas las fechas son sintácticamente correctas.

In [4]:
import re
pattern = re.compile("[\s]?[0-9][0-9][0-9][0-9][-][0-1][0-9][-][0-3][0-9]\s[0-2][0-9][:][0-5][0-9][:][0-5][0-9]")


def preprocesa(x):
    listaFechas = x.split(",")
    for date in listaFechas:
        if(pattern.match(date) == None):
            print("date not matching: " + str(x))
            return True
    return False

df_check['fecha'].apply(lambda x: preprocesa(x))

0         False
1         False
2         False
3         False
4         False
          ...  
161945    False
161946    False
161947    False
161948    False
161949    False
Name: fecha, Length: 161950, dtype: bool